In [27]:
import pandas as pd
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import time
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)
import tweepy

In [2]:
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

## Scraping

#### NASA Mars News

Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [6]:
news_title = soup.find("div", class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text
print(news_title)
print(news_p)

Third ASPIRE Test Confirms Mars 2020 Parachute a Go
The supersonic parachute that will handle the heaviest payload yet to the Red Planet – Mars 2020 rover – passes its final sounding rocket test with flying colors.


#### JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.
- Make sure to find the image url to the full size `.jpg` image.
- Make sure to save a complete url string for this image.

In [23]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
time.sleep(5)

browser.click_link_by_partial_text("FULL IMAGE")
time.sleep(5)
browser.click_link_by_partial_text("more info")
time.sleep(5)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

results = soup.find("article")
extension = results.find("figure", "lede").a["href"]
base_url = "https://www.jpl.nasa.gov"
featured_image_url = base_url + extension
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16815_hires.jpg'

#### Mars Weather

Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [28]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [29]:
target_user = "@MarsWxReport"

In [31]:
tweet = api.user_timeline(target_user)[0]
mars_weather = tweet["text"]
mars_weather

'Sol 2209 (2018-10-23), high -18C/0F, low -73C/-99F, pressure at 8.79 hPa, daylight 06:09-18:26'

#### Mars Facts

- Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
- Use Pandas to convert the data to a HTML table string.

In [33]:
url = "http://space-facts.com/mars/"
mars_facts = pd.read_html(url)
mars_facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [36]:
mars_facts_df = mars_facts[0]
mars_facts_df.columns=["Facet", "Value"]

mars_facts_df['Facet'] = mars_facts_df['Facet'].map(lambda x: x.rstrip(':').lstrip('aAbBcC'))
mars_facts_df

,Facet,Value
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.42 x 10^23 kg (10.7% Earth)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.52 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-153 to 20 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [43]:
mars_facts_table = mars_facts_df.to_html()
mars_facts_table.replace("\n", "")
print(mars_facts_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Facet</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorde

#### Mars Hemispheres

- Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.
- You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.
- Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [45]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

hemisphere_image_urls = []

In [47]:
# Cerberus
browser.visit(hemi_url)
time.sleep(5)
browser.click_link_by_partial_text("Cerberus Hemisphere Enhanced")

html = browser.html
soup = BeautifulSoup(html, "html.parser")

cerberus_link = soup.find("div", "downloads").a["href"]

cerberus = {
    "title": "Cerberus Hemisphere",
    "img_url": cerberus_link
}

hemisphere_image_urls.append(cerberus)

In [49]:
# Schiaparelli 
browser.visit(hemi_url)
time.sleep(5)
browser.click_link_by_partial_text("Schiaparelli Hemisphere Enhanced")

html = browser.html
soup = BeautifulSoup(html, "html.parser")

schiaparelli_link = soup.find("div", "downloads").a["href"]

schiaparelli = {
    "title": "Cerberus Hemisphere",
    "img_url": schiaparelli_link
}

hemisphere_image_urls.append(schiaparelli)

In [52]:
# Syrtis Major
browser.visit(hemi_url)
time.sleep(5)
browser.click_link_by_partial_text("Syrtis Major Hemisphere Enhanced")

html = browser.html
soup = BeautifulSoup(html, "html.parser")

syrtis_major_link = soup.find("div", "downloads").a["href"]

syrtis_major = {
    "title": "Cerberus Hemisphere",
    "img_url": syrtis_major_link
}

hemisphere_image_urls.append(syrtis_major)

In [53]:
# Valles Marineris
browser.visit(hemi_url)
time.sleep(5)
browser.click_link_by_partial_text("Valles Marineris Hemisphere Enhanced")

html = browser.html
soup = BeautifulSoup(html, "html.parser")

valles_marineris_link = soup.find("div", "downloads").a["href"]

valles_marineris = {
    "title": "Cerberus Hemisphere",
    "img_url": valles_marineris_link
}

hemisphere_image_urls.append(valles_marineris)

In [54]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]